In [78]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn import preprocessing

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import re

from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from functools import reduce
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy3
from pymystem3 import Mystem


import warnings
warnings.filterwarnings('ignore')

In [79]:
df2 = pd.DataFrame(columns=['text', 'label'])
df1 = pd.DataFrame(columns=['text', 'label'])
df_lem_train = pd.DataFrame(columns=['text', 'label'])
df_lem_test = pd.DataFrame(columns=['text', 'label'])

cou = 0

# достаем обучающую выборку и загружаем в 2 ДатаФрейма: для лемматизации и стемминга
df_train = pd.read_csv('/Users/mac/Downloads/rusentitweet_train.csv')
for i, row in df_train.iterrows():
    if row[1] in ("negative", "positive"):
        if row[1]=='negative':
            cou+=1
        df2.loc[len(df2.index)] = row
        df_lem_train.loc[len(df_lem_train.index)] = row
        
df_train = df2
print('Процент negative в train: ', cou/df_train.shape[0])
        
cou = 0
# достаем тестову выборку и загружаем в 2 ДатаФрейма: для лемматизации и стемминга
df_test = pd.read_csv('/Users/mac/Downloads/rusentitweet_test.csv')
for i, row in df_test.iterrows():
    if row[1] in ("negative", "positive"):
        df1.loc[len(df1.index)] = row
        if row[1]=='negative':
            cou+=1
        df_lem_test.loc[len(df_lem_test.index)] = row
df_test = df1
print('Процент negative в test: ', cou/df_test.shape[0])


print(df_train)

Процент negative в train:  0.5773692274020573
Процент negative в test:  0.5774278215223098
                                                   text     label
0     Помойму я вкрашилась в Чимина🤧 https://t.co/t2...  positive
1                         @buybread_ я не с порядке!!!!  negative
2     @ange1flyhigh В следующий раз буду до победног...  positive
3     @LimitaVIP Удивительный гiмн...\r\nУдивительно...  negative
4                               я срала на эту биологию  negative
...                                                 ...       ...
4564  Дедрадио5 то есть ты хочешь сказать, что ты ла...  negative
4565  @kmoo_m ДЕЙСТВИТЕЛЬНО\r\nесли любовь, то тольк...  positive
4566  С Хоби утро может быть только добрым😂 https://...  positive
4567  Не успела встать, а уже заебалась, впрочем нич...  negative
4568  ах твиттер ивана урганта, а ведь всё так хорош...  negative

[4569 rows x 2 columns]


In [80]:
nltk.download('stopwords')
# Стоп-слова
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
# 2 Функция очистки
def my_clear(df):
    s=[]
    s1 = ""
    regs = [r'https?:\/\/.\S+', r'#\S+', r'@\S+', r'\\r\\n', r'[-.)(,:]']
    glasn = ['о', 'а', 'я', 'а', 'и', 'е', 'ы']
    signs = ['!', '_', '.']

    # приведение всего к нижнему регистру
    for i, row in df.iterrows():
        row[0] = row[0].lower()


    for i, row in df.iterrows():
        # удаление ссыло, хэштегов(и знака решетки и последующего хэштега, потому что слова после хэштэга пишутся слитно)
        # удаление отмеченных пользователей(@name), удаление знаков пунктуации (кроме ! ?)
        for reg in regs:
            row[0] = re.sub(reg, "", row[0])

        # удаление стоп-слов
        row[0] = ' '.join([x for x in row[0].split() if x not in stop])

        # удаление повторящихся подряд смайликов и знаков препинания (оставшихся)
        # удаление подряд идущих гласных (тыыыыы, яяяяяя, аааааа и тд)
        # удаление цифр, так как по сути цифры не имеют никакого значения в данном анализе
        r = row[0]
        for t in range(0, len(row[0])):
            if not(not(r[t].isalpha()) and r[t] == r[t-1]) and not((r[t] in glasn) and r[t] == r[t-1]) and not(r[t].isdigit()):
                s1+=r[t]
        row[0] = s1

        s1 = ""

        r = row[0] + "  "

        #отделение смайлика от слова (чимина🤧 -> чимина 🤧)
        for t in range(0, len(row[0])):
            if not(r[t+1].isalpha()) and r[t]!= " " and not(r[t+1].isdigit()) and (r[t+2].isalpha() or r[t+2]==" "):
                s1+=r[t] + " "
            else:
                s1+=r[t]
        row[0] = s1

        s = []
        s1=""
    return df

In [82]:
# 2 очистка данных
df_train = my_clear(df_train)
df_test = my_clear(df_test)

In [83]:
# 3
# стемминг обучающих и тестовых
new = SnowballStemmer(language='russian')
for i, row in df_test.iterrows():
    row[0] = ' '.join([new.stem(x) for x in row[0].split()])
    
for i, row in df_train.iterrows():
    row[0] = ' '.join([new.stem(x) for x in row[0].split()])

In [84]:
x_train = []
x_test = []

# создание массива строк
for i, row in df_train.iterrows():
    x_train.append(row[0])

for i, row in df_test.iterrows():
    x_test.append(row[0])
x_train[0:10]

['помойм вкраш чимин 🤧',
 'порядк !',
 'след буд победн закрыва пожела удач !',
 'удивительн гимн удивительн пок ещ сдохл украин',
 'срал биолог',
 'помим алин ещ радост гемоглобин повыс',
 'пиздец че вобщ чувств жизн помота прошл сутк туман',
 'спичк ? зажеч огон тво глаз',
 'эт сам дел очен крут',
 'хоч сказа чтот приятн получа "ид нах ебла противн "']

In [119]:
# 4
vec = TfidfVectorizer(stop_words=None, min_df=0.0, ngram_range=(1,4))

# tf-idf test and train
matr = vec.fit_transform(x_train)
matr_test = vec.transform(x_test)


In [86]:
c = {}
train = {}

for i in np.arange(0.01, 4, 0.1):
    model_logreg = LogisticRegression(C=i)
    model_logreg.fit(matr, df_train['label'])
    y_logreg = model_logreg.predict(matr_test)
    model_logreg.fit(matr, df_train['label'])
    accuracy = accuracy_score(df_test['label'], y_logreg)
    y_logreg = model_logreg.predict(matr)
    accuracy1 = accuracy_score(df_train['label'], y_logreg)
    
    c[i] = accuracy 
    train[i] = accuracy1
    
c = sorted(c.items(), key=lambda x: x[1])
c = c[::-1]
train = sorted(train.items(), key=lambda x: x[1])
train = train[::-1]


print(c)
print(train)

[(3.91, 0.773403324584427), (3.81, 0.773403324584427), (3.7100000000000004, 0.773403324584427), (3.6100000000000003, 0.7716535433070866), (3.21, 0.7707786526684165), (3.41, 0.7699037620297463), (3.31, 0.7699037620297463), (2.91, 0.7699037620297463), (3.5100000000000002, 0.7690288713910761), (3.1100000000000003, 0.7690288713910761), (3.0100000000000002, 0.7690288713910761), (2.81, 0.7690288713910761), (2.6100000000000003, 0.7690288713910761), (2.71, 0.7681539807524059), (2.5100000000000002, 0.7681539807524059), (2.41, 0.7664041994750657), (2.31, 0.7646544181977253), (2.1100000000000003, 0.7646544181977253), (1.81, 0.7646544181977253), (2.21, 0.7629046369203849), (2.0100000000000002, 0.7629046369203849), (1.9100000000000001, 0.7620297462817148), (1.7100000000000002, 0.7611548556430446), (1.51, 0.7602799650043744), (1.61, 0.7585301837270341), (1.4100000000000001, 0.7567804024496938), (1.31, 0.7567804024496938), (1.21, 0.7532808398950132), (0.81, 0.7532808398950132), (1.11, 0.7524059492563

In [120]:
# 5 logreg
model_logreg = LogisticRegression(C=.31)
model_logreg.fit(matr, df_train['label'])
y_logreg = model_logreg.predict(matr)

accuracy = accuracy_score(df_train['label'], y_logreg)
print(accuracy)

0.7089078572991901


In [121]:
model_logreg = LogisticRegression(C=0.31)
model_logreg.fit(matr, df_train['label'])
y_logreg = model_logreg.predict(matr_test)

accuracy = accuracy_score(df_test['label'], y_logreg)
print(accuracy)

0.7147856517935258


In [122]:
# 5 top logreg
importance = model_logreg.coef_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()
sorted_index = sorted_index[0][::-1]
# топ-10 значимых слов (горячая десятка)
np.array(vec.get_feature_names_out())[sorted_index[0:10]]

array(['любл', 'красив', 'блят', 'пиздец', 'мил', 'крут', 'ва',
       'прекрасн', 'лучш', 'хорош'], dtype=object)

In [90]:
c = {}
train = {}

for i in range(25, 55):
    model_random_forest = RandomForestClassifier(max_depth=i)
    model_random_forest.fit(matr, df_train['label'])
    y_logreg = model_random_forest.predict(matr_test)
    model_random_forest.fit(matr, df_train['label'])
    accuracy = accuracy_score(df_test['label'], y_logreg)
    y_logreg = model_random_forest.predict(matr)
    accuracy1 = accuracy_score(df_train['label'], y_logreg)
    
    c[i] = accuracy 
    train[i] = accuracy1
    
c = sorted(c.items(), key=lambda x: x[1])
c = c[::-1]
train = sorted(train.items(), key=lambda x: x[1])
train = train[::-1]


print(c)
print(train)

[(48, 0.6867891513560805), (54, 0.6850393700787402), (53, 0.6824146981627297), (47, 0.6806649168853893), (52, 0.6780402449693789), (51, 0.6745406824146981), (50, 0.673665791776028), (46, 0.673665791776028), (49, 0.6719160104986877), (45, 0.6710411198600175), (44, 0.6710411198600175), (42, 0.6701662292213474), (37, 0.6692913385826772), (38, 0.6640419947506562), (39, 0.6587926509186351), (40, 0.657917760279965), (41, 0.6570428696412949), (36, 0.6570428696412949), (43, 0.6552930883639545), (32, 0.6465441819772528), (31, 0.6447944006999126), (34, 0.6404199475065617), (35, 0.6351706036745407), (33, 0.6342957130358705), (30, 0.6334208223972003), (26, 0.6307961504811899), (28, 0.6229221347331584), (29, 0.615923009623797), (27, 0.6132983377077865), (25, 0.6115485564304461)]
[(54, 0.7393302692055155), (53, 0.7380170715692712), (51, 0.7301378857518056), (52, 0.7277303567520245), (50, 0.7211643685708032), (49, 0.7178813744801926), (46, 0.7172247756620704), (48, 0.7167870431166558), (47, 0.7128474

In [129]:
# 5 random_forest
model_random_forest = RandomForestClassifier(max_depth=35)
model_random_forest.fit(matr, df_train['label'])
y_forest = model_random_forest.predict(matr)

accuracy = accuracy_score(df_train['label'], y_forest)
print(accuracy)

0.6596629459400306


In [130]:
model_random_forest = RandomForestClassifier(max_depth=35)
model_random_forest.fit(matr, df_train['label'])
y_forest = model_random_forest.predict(matr_test)

accuracy = accuracy_score(df_test['label'], y_forest)
print(accuracy)

0.6622922134733158


In [131]:
# 5 top random_forest
importance = model_random_forest.feature_importances_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()[::-1]
# топ-10 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:10]]

array(['красив', 'лучш', 'блят', 'пиздец', 'мил', 'любл', 'прекрасн',
       'крут', 'хорош', 'нах'], dtype=object)

In [94]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mac/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [138]:
# clear
df_lem_train = my_clear(df_lem_train)
df_lem_test = my_clear(df_lem_test)

In [139]:
# Mystem - контекстный лемматизатор + pip install pymorphy3-dicts-ru - словари
m = Mystem()

x_test_lem = []
x_train_lem = []

# lemmatize
for i, t in df_lem_train.iterrows():
    lemmas = m.lemmatize(t[0])
    x_train_lem.append("".join(lemmas).strip())
    
for i, t in df_lem_test.iterrows():
    lemmas = m.lemmatize(t[0])
    x_test_lem.append("".join(lemmas).strip())

In [140]:
vec = TfidfVectorizer(stop_words=None)

# tf-idf test and train
matr_train_lem = vec.fit_transform(x_train_lem)
matr_test_lem = vec.transform(x_test_lem)

In [102]:
c = {}
train = {}

for i in np.arange(0.01, 4, 0.1):
    model_logreg = LogisticRegression(C=i)
    model_logreg.fit(matr_train_lem, df_train['label'])
    y_logreg = model_logreg.predict(matr_test_lem)
    model_logreg.fit(matr_train_lem, df_train['label'])
    accuracy = accuracy_score(df_test['label'], y_logreg)
    y_logreg = model_logreg.predict(matr_train_lem)
    accuracy1 = accuracy_score(df_train['label'], y_logreg)
    
    c[i] = accuracy 
    train[i] = accuracy1
    
c = sorted(c.items(), key=lambda x: x[1])
c = c[::-1]
train = sorted(train.items(), key=lambda x: x[1])
train = train[::-1]


print(c)
print(train)

[(2.31, 0.7865266841644795), (3.0100000000000002, 0.7856517935258093), (2.91, 0.7856517935258093), (2.6100000000000003, 0.7856517935258093), (2.41, 0.7856517935258093), (3.6100000000000003, 0.7847769028871391), (3.31, 0.7847769028871391), (3.21, 0.7847769028871391), (2.81, 0.7847769028871391), (2.71, 0.7847769028871391), (2.5100000000000002, 0.7847769028871391), (3.81, 0.7839020122484689), (3.7100000000000004, 0.7839020122484689), (3.41, 0.7839020122484689), (3.1100000000000003, 0.7839020122484689), (2.21, 0.7839020122484689), (1.9100000000000001, 0.7839020122484689), (3.91, 0.7830271216097988), (3.5100000000000002, 0.7830271216097988), (2.1100000000000003, 0.7830271216097988), (2.0100000000000002, 0.7830271216097988), (1.7100000000000002, 0.7830271216097988), (1.81, 0.7821522309711286), (1.61, 0.7804024496937882), (1.4100000000000001, 0.7795275590551181), (1.51, 0.778652668416448), (1.31, 0.778652668416448), (1.11, 0.7769028871391076), (1.01, 0.7769028871391076), (1.21, 0.776027996500

In [141]:
model_logreg = LogisticRegression(C=0.21)
model_logreg.fit(matr_train_lem, df_lem_train['label'])
y_logreg = model_logreg.predict(matr_train_lem)

accuracy = accuracy_score(df_lem_train['label'], y_logreg)
print(accuracy)

0.7255416940249507


In [142]:
model_logreg = LogisticRegression(C=0.21)
model_logreg.fit(matr_train_lem, df_lem_train['label'])
y_logreg = model_logreg.predict(matr_test_lem)

accuracy = accuracy_score(df_lem_test['label'], y_logreg)
print(accuracy)

0.7086614173228346


In [143]:
importance = model_logreg.coef_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()
sorted_index = sorted_index[0][::-1]
# топ-20 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:10]]

array(['любить', 'блять', 'хороший', 'красивый', 'пиздец', 'вау', 'милый',
       'умирать', 'нравиться', 'сука'], dtype=object)

In [111]:
c = {}
train = {}

for i in range(25, 55):
    model_random_forest = RandomForestClassifier(max_depth=i)
    model_random_forest.fit(matr_train_lem, df_train['label'])
    y_logreg = model_random_forest.predict(matr_test_lem)
    model_random_forest.fit(matr_train_lem, df_train['label'])
    accuracy = accuracy_score(df_test['label'], y_logreg)
    y_logreg = model_random_forest.predict(matr_train_lem)
    accuracy1 = accuracy_score(df_train['label'], y_logreg)
    
    c[i] = accuracy 
    train[i] = accuracy1
    
c = sorted(c.items(), key=lambda x: x[1])
c = c[::-1]
train = sorted(train.items(), key=lambda x: x[1])
train = train[::-1]


print(c)
print(train)

[(52, 0.7226596675415573), (40, 0.7217847769028871), (54, 0.7200349956255468), (51, 0.7165354330708661), (50, 0.7147856517935258), (48, 0.7130358705161854), (45, 0.710411198600175), (43, 0.710411198600175), (42, 0.710411198600175), (41, 0.710411198600175), (49, 0.7095363079615048), (47, 0.7086614173228346), (46, 0.7086614173228346), (53, 0.7069116360454943), (39, 0.705161854768154), (37, 0.705161854768154), (44, 0.7034120734908137), (38, 0.6990376202974629), (32, 0.6990376202974629), (33, 0.6964129483814523), (36, 0.6955380577427821), (35, 0.6937882764654418), (34, 0.6920384951881015), (28, 0.6902887139107612), (29, 0.68416447944007), (30, 0.6832895888013998), (25, 0.6824146981627297), (31, 0.6806649168853893), (26, 0.6719160104986877), (27, 0.6657917760279966)]
[(54, 0.7793827971109653), (53, 0.7785073320201357), (52, 0.7769752681111841), (51, 0.7747866053841103), (50, 0.7728168089297439), (49, 0.7688772160210111), (47, 0.7671262858393522), (48, 0.7666885532939374), (46, 0.76603195447

In [144]:
model_random_forest = RandomForestClassifier(max_depth=25)
model_random_forest.fit(matr_train_lem, df_lem_train['label'])
y_forest = model_random_forest.predict(matr_test_lem)

accuracy = accuracy_score(df_lem_test['label'], y_forest)
print(accuracy)

0.6745406824146981


In [145]:
model_random_forest = RandomForestClassifier(max_depth=25)
model_random_forest.fit(matr_train_lem, df_lem_train['label'])
y_forest = model_random_forest.predict(matr_test_lem)

accuracy = accuracy_score(df_lem_test['label'], y_forest)
print(accuracy)

0.6640419947506562


In [146]:
importance = model_random_forest.feature_importances_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()[::-1]
# топ-10 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:10]]

array(['любить', 'блять', 'хороший', 'красивый', 'вау', 'пиздец', 'милый',
       'умирать', 'нахуй', 'сука'], dtype=object)